In [71]:
import backtrader as bt
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [5]:
class PelosiStrategy(bt.Strategy):
    params = (
        ('delay_days', 1),  # Days to wait after Pelosi's trade
        ('position_size', 0.05),  # 5% of portfolio per trade
    )

    def __init__(self):
        self.trades_df = pd.read_csv('./data/nancy.tsv', delimiter='\t')

        if 'Transaction Date' in self.trades_df.columns:
            self.trades_df = self.trades_df.rename(columns={
                'Transaction Date': 'Date',
                'Full Name': 'Ticker',
                'Ticker': 'Name'
            })

        self.trades_df['Date'] = pd.to_datetime(self.trades_df['Date'])

        self.purchases = self.trades_df[self.trades_df['Transaction'] == 'Purchase'].copy()
        self.purchases = self.purchases.sort_values('Date')

        self.signals = []
        for _, trade in self.purchases.iterrows():
            signal_date = trade['Date'] + timedelta(days=self.params.delay_days)
            self.signals.append({
                'date': signal_date,
                'ticker': trade['Ticker'],
                'original_date': trade['Date']
            })

        self.signals = sorted(self.signals, key=lambda x: x['date'])
        self.signal_index = 0

        print(f"Loaded {len(self.signals)} buy signals")

    def next(self):
        current_date = self.data.datetime.date(0)

        while (self.signal_index < len(self.signals) and
               self.signals[self.signal_index]['date'].date() <= current_date):

            signal = self.signals[self.signal_index]
            ticker = signal['ticker']

            data_feed = None
            for i, d in enumerate(self.datas):
                if hasattr(d, '_name') and d._name == ticker:
                    data_feed = d
                    break

            if data_feed is not None:
                portfolio_value = self.broker.getvalue()
                position_value = portfolio_value * self.params.position_size

                price = data_feed.close[0]
                shares = int(position_value / price)

                if shares > 0 and self.broker.getcash() >= shares * price:
                    self.buy(data=data_feed, size=shares)
                    print(f"{current_date}: BUY {shares} shares of {ticker} @ ${price:.2f} "
                          f"(Pelosi trade: {signal['original_date'].date()})")

            self.signal_index += 1



In [63]:
trades_df = pd.read_csv('./data/nancy.tsv', delimiter='\t')

if 'Transaction Date' in trades_df.columns:
    trades_df = trades_df.rename(columns={
        'Transaction Date': 'Date',
        'Full Name': 'Ticker',
        'Ticker': 'Name'
    })

trades_df['Date'] = pd.to_datetime(trades_df['Date'])
purchases = trades_df[trades_df['Transaction'] == 'Purchase']

tickers = purchases['Ticker'].unique().tolist()
print(f"Found {len(tickers)} unique tickers")

cerebro = bt.Cerebro()
cerebro.addstrategy(PelosiStrategy)
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)  # 0.1% commission

start_date = datetime(2018, 1, 1)
end_date = datetime.now()

successful_tickers = []

cache_dir = "./cache"
os.makedirs(cache_dir, exist_ok=True)

for ticker in tickers[:20]:
    try:
        cache_file = os.path.join(cache_dir, f"{ticker}.csv")

        if os.path.exists(cache_file):
            stock_data = pd.read_csv(cache_file)
            print(f"✓ Loaded {ticker} from cache")
        else:
            stock_data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
            if not stock_data.empty:
                stock_data.to_csv(cache_file)
                print(f"✓ Downloaded and cached {ticker}")
            else:
                print(f"✗ No data for {ticker}")
                continue

        stock_data = stock_data.drop([0,1], axis=0)
        stock_data = stock_data.rename(columns={"Price": "Date"})
        # stock_data = stock_data.set_index("Date")
        stock_data[["Open", "High", "Low", "Close", "Volume"]] = stock_data[["Open", "High", "Low", "Close", "Volume"]].apply(pd.to_numeric)
        print(stock_data.head())
        stock_data["Date"] = stock_data["Date"].apply(lambda x: pd.to_datetime(x, format="%Y-%m-%d"))

        data = bt.feeds.PandasData(
            dataname=stock_data,
            datetime="Date",
            open='Open',
            high='High',
            low='Low',
            close='Close',
            volume='Volume',
            openinterest=None
        )
        data._name = ticker  # for identification
        cerebro.adddata(data)
        successful_tickers.append(ticker)

    except Exception as e:
        print(f"✗ Error loading {ticker}: {str(e)}")

print(f"\nSuccessfully loaded {len(successful_tickers)} tickers")




Found 27 unique tickers
✓ Loaded AVGO from cache
         Date      Close       High        Low       Open    Volume
2  2018-01-02  21.207603  21.246522  20.457819  20.632557  33135000
3  2018-01-03  21.439522  21.503063  21.129760  21.229837  31161000
4  2018-01-04  21.446674  21.584876  21.320386  21.571373  19067000
5  2018-01-05  21.573753  21.783439  21.401398  21.601552  28633000
6  2018-01-08  21.625383  21.647623  21.370426  21.518158  20971000
✓ Loaded AMZN from cache
         Date      Close       High        Low       Open    Volume
2  2018-01-02  59.450500  59.500000  58.525501  58.599998  53890000
3  2018-01-03  60.209999  60.274502  59.415001  59.415001  62176000
4  2018-01-04  60.479500  60.793499  60.233002  60.250000  60442000
5  2018-01-05  61.457001  61.457001  60.500000  60.875500  70894000
6  2018-01-08  62.343498  62.653999  61.601501  61.799999  85590000
✓ Loaded GOOGL from cache
         Date      Close       High        Low       Open    Volume
2  2018-01-02  5

In [83]:
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')           # Total and annualized returns
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')        # Sharpe Ratio
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')         # Maximum and average drawdown
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')      # Trade-level metrics (win/loss, profit/loss)
cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')                   # System Quality Number, related to risk/reward per trade
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='annual_return')# Annualized returns

In [84]:
from datetime import datetime, timedelta

print(f'\nStarting Portfolio Value: {cerebro.broker.getvalue():.2f}')
results = cerebro.run()


Starting Portfolio Value: 119580.84
Loaded 95 buy signals
2024-06-14: BUY 50 shares of DIS @ $98.65 (Pelosi trade: 2014-11-28)
2024-06-14: BUY 50 shares of DIS @ $98.65 (Pelosi trade: 2014-12-10)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2016-01-13)
2024-06-14: BUY 50 shares of DIS @ $98.65 (Pelosi trade: 2016-01-15)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2016-05-17)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2017-01-19)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2017-06-15)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-02-02)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-02-02)
2024-06-14: BUY 27 shares of AMZN @ $183.66 (Pelosi trade: 2018-07-27)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-09-11)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-09-20)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-09-20)
2024-06-14: BUY 27 share

In [85]:
print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')
strat = results[0]
# cerebro.plot(style='candlestick')


Final Portfolio Value: 119580.84


In [86]:
results = cerebro.run()
strat = results[0]

# Extract analyzers
analyzers = {
    'Returns': strat.analyzers.returns.get_analysis(),
    'Sharpe': strat.analyzers.sharpe.get_analysis(),
    'Drawdown': strat.analyzers.drawdown.get_analysis(),
    'Trades': strat.analyzers.trades.get_analysis(),
    'Calmar': strat.analyzers.calmar.get_analysis(),
    'Annual': strat.analyzers.annual_return.get_analysis(),
    'SQN': strat.analyzers.sqn.get_analysis()
}

Loaded 95 buy signals
2024-06-14: BUY 50 shares of DIS @ $98.65 (Pelosi trade: 2014-11-28)
2024-06-14: BUY 50 shares of DIS @ $98.65 (Pelosi trade: 2014-12-10)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2016-01-13)
2024-06-14: BUY 50 shares of DIS @ $98.65 (Pelosi trade: 2016-01-15)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2016-05-17)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2017-01-19)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2017-06-15)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-02-02)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-02-02)
2024-06-14: BUY 27 shares of AMZN @ $183.66 (Pelosi trade: 2018-07-27)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-09-11)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-09-20)
2024-06-14: BUY 23 shares of AAPL @ $211.26 (Pelosi trade: 2018-09-20)
2024-06-14: BUY 27 shares of AMZN @ $183.66 (Pelosi trade: 20

In [103]:
print("\n=== Returns ===")
print(f"Total Return: {analyzers['Returns']['rtot']*100:.2f}%")
print(f"CAGR: {analyzers['Returns']['rnorm']*100:.2f}%")
print("-- Yearly ---")
for year, value in analyzers['Annual'].items():
    print(f"{year}: {value*100:.2f}%")

print("\n=== Risk-Adjusted Metrics ===")
print(f"Sharpe Ratio: {analyzers['Sharpe']['sharperatio']:.2f}")
print(f"SQN: {analyzers['SQN']['sqn']:.2f}")
# print(f"Sortino: {sortino_ratio:.2f}%")

print("\n=== Drawdown ===")
print(f"Max Drawdown: {analyzers['Drawdown']['max']['drawdown']:.2f}%")
print(f"Drawdown Duration: {analyzers['Drawdown']['max']['len']} periods")

print("\n=== Trades ===")
print(analyzers["Trades"])




=== Returns ===
Total Return: 17.88%
CAGR: 2.37%
-- Yearly ---
2018: 0.00%
2019: 0.00%
2020: 0.00%
2021: 0.00%
2022: 0.00%
2023: 0.00%
2024: 20.82%
2025: -1.02%

=== Risk-Adjusted Metrics ===
Sharpe Ratio: 0.21
SQN: 0.00

=== Drawdown ===
Max Drawdown: 26.99%
Drawdown Duration: 167 periods

=== Trades ===
AutoOrderedDict({'total': AutoOrderedDict({'total': 5, 'open': 5})})
